In [1]:
import tensorflow as tf

In [2]:
import tensorflow.contrib.learn.python.learn as learn
from sklearn import datasets, metrics

In [3]:
import pandas as pd

import sklearn.cross_validation as cv

/projects/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:

# Data is obtained at https://archive.ics.uci.edu/ml/datasets/Pima+Indians+Diabetes
dataset=pd.read_csv("pima_b.csv", header=None)

In [5]:
# define features and labels
X=dataset.iloc[:, 0:8]
y=dataset.iloc[:, -1]


In [6]:
from sklearn import preprocessing as pre
scaler = pre.StandardScaler().fit(X)

In [7]:
X_scaled = scaler.transform(X)

In [9]:
import numpy as np

seed = 7
np.random.seed(seed)

In [10]:
# determine train/test split
x_train, x_test, y_train, y_test = cv.train_test_split(X_scaled, 
                                                       y, 
                                                       test_size=1.0/3, 
                                                       random_state=0)

In [11]:
feature_columns = learn.infer_real_valued_columns_from_input(x_train)

In [12]:
# Define a standard 3 layer model
classifier = learn.DNNClassifier(hidden_units=[10, 20, 10], n_classes=2, feature_columns=feature_columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_evaluation_master': '', '_task_type': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f49dbb0ecc0>, '_environment': 'local', 'keep_checkpoint_every_n_hours': 10000, '_is_chief': True, '_num_ps_replicas': 0, '_master': '', '_task_id': 0, 'save_checkpoints_secs': 600, 'save_summary_steps': 100, 'save_checkpoints_steps': None, 'tf_random_seed': None, 'keep_checkpoint_max': 5, 'tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
}


In [13]:
# fit the model
classifier.fit(x_train, y_train, batch_size=128, steps=100)

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Summary name dnn/hiddenlayer_0:fraction_of_zero_values is illegal; using

In [14]:
#evaluate the model
predictions = list(classifier.predict(x_test, as_iterable=True))

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Summary name dnn/hiddenlayer_0:fraction_of_zero_values is illegal; using

In [15]:
score = metrics.accuracy_score(y_test,predictions)

In [16]:
print("Accuracy: %f" % score)

Accuracy: 0.785156


In [22]:
import tensorflow.contrib.layers.python.layers as layers

In [23]:
# define a custom model of 3 hidden layers
def my_model(features, labels):
    # Convert the labels to a one-hot tensor of shape
    
    labels = tf.one_hot(labels, 2, 1, 0)
    features = layers.stack(features, layers.fully_connected, [10, 20, 10])
    
    # Compute logits (1 per class) and compute loss.
    logits = layers.fully_connected(features, 2, activation_fn=tf.nn.softmax)
    loss = tf.contrib.losses.softmax_cross_entropy(logits, labels)   
    
    # Create a tensor for training op
    train_op = tf.contrib.layers.optimize_loss(loss,
       tf.contrib.framework.get_global_step(), optimizer='Adam',learning_rate=0.1)
    
    return {'class': tf.argmax(logits, 1), 'prob': tf.nn.softmax(logits)}, loss, train_op
    

In [24]:
classifier = learn.Estimator(model_fn=my_model)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_evaluation_master': '', '_task_type': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f49c0e4b780>, '_environment': 'local', 'keep_checkpoint_every_n_hours': 10000, '_is_chief': True, '_num_ps_replicas': 0, '_master': '', '_task_id': 0, 'save_checkpoints_secs': 600, 'save_summary_steps': 100, 'save_checkpoints_steps': None, 'tf_random_seed': None, 'keep_checkpoint_max': 5, 'tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
}


In [25]:
classifier.fit(x_train, y_train, steps=1000)

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:loss = 0.730013132095, step = 1
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp5ln7qq1i/model.ckpt.
INFO:tensorflow:loss = 0.489996194839, step = 101
INFO:tensorflow:global_step/sec: 62.3987
INFO:tensorflow:loss = 0.511645019054, step = 201
INFO:tensorflow:global_step/sec: 88.6855
INFO:tensorflow:loss = 0.508657217026, step

Estimator(params=None)

In [26]:
# obtain predicted labels
y_predicted = [
  p['class'] for p in classifier.predict(x_test, as_iterable=True)]

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Loading model from checkpoint: /tmp/tmp5ln7qq1i/model.ckpt-1000-?????-of-00001.


In [27]:
# determine if the score is improved
score = metrics.accuracy_score(y_test, y_predicted)

In [28]:
print('Accuracy: {0:f}'.format(score))

Accuracy: 0.753906
